# Treatment of problematic documents

In [ ]:
import subprocess
import os

path = "C:/Users/Marine/Documents/CO_uniandes/0_Tesis/MT_papers/WOI"
new_path = path.replace('/', '\\')

for picture in os.listdir(path):
    if '.png' in picture:
        new = picture.replace('.png', '')
        command = ["tesseract", picture, new]
        result = subprocess.run(["wsl"] + command, cwd=new_path, capture_output=True)

In [ ]:
import os

file_path = "C:/Users/Marine/Documents/CO_uniandes/0_Tesis/MT_papers/WOI"

new_content = ''
for file in os.listdir(file_path):
    try:
        if '.txt' in file:
            with open(file_path+'/'+file, 'r') as f:
                contents = f.read()
                lines = contents.split('\n')
                prev_line_empty = False
                
                for line in lines:
                    if line.strip():
                        if prev_line_empty:
                            new_content+='\n'
                        new_content+=line
                        prev_line_empty=False
                    else:
                        prev_line_empty = True
                new_content+='\n'
                        
                #print(new_content)
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except Exception as e:
        print(f"An error occurred on {f}: {e}")
        
print(new_content)
with open(file_path+'/final/output.txt', 'w') as f:
    f.write(new_content)

# Embeddings

In [20]:
import os
import time
import pandas as pd
import numpy as np
import openai
import tiktoken
import PyPDF2

openai.api_key = "cf0bd49030ed4aa6a6509be1cd9d604b"
openai.api_base = "https://invuniandesai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15'
tokenizer = tiktoken.get_encoding("cl100k_base")

def process_to_txt():

    input_directory = "C:/Users/Marine/Documents/CO_uniandes/0_Tesis/MT_papers/WOI/"
    files=[]
    pages=[]
    raw_text = []
    ok_docs=0
    
    for root, dirs, filenames in os.walk(input_directory):
        for file in filenames:
            if '.pdf' in file:
                # Ouvrir le fichier PDF en mode lecture binaire ('rb')
                with open(root+'/'+file, 'rb') as pdf_file:
                    # Créer un objet PDFReader
                    try:
                        pdf_reader = PyPDF2.PdfReader(pdf_file)

                        # Parcourir chaque page du PDF
                        for page_num in range(len(pdf_reader.pages)):
                            # Extraire le texte de la page
                            files.append(file.split('.pdf')[0])
                            pages.append(page_num)
                            raw_text.append(pdf_reader.pages[page_num].extract_text())
                        ok_docs+=1
                    except Exception as e:
                        print(pdf_file," ", e)
            else : 
                try: 
                    with open(root+'/'+file, 'r') as txt_file:
                        files.append(file.split('.txt')[0])
                        pages.append('1')
                        raw_text = txt_file.read()
                    ok_docs+=1
                except Exception as e:
                        print(txt_file," ", e)
                    
                    
    
    print(ok_docs)                
    data = {'name': files, 'page': pages, 'raw_text': raw_text}
    df = pd.DataFrame(data)
    
    df['n_tokens'] = df.raw_text.apply(lambda x: len(tokenizer.encode(x)))
    return df
                
max_tokens = 1500

# Function to split the text into chunks of a maximum number of tokens
def split_into_many(text, file_name, pages, max_tokens = max_tokens):
    
    # Split the text into paragraphs
    paragraphs = text.split('\n\n')
    
    # Get the number of tokens for each paragraph
    n_tokens = [len(tokenizer.encode(" " + paragraph)) for paragraph in paragraphs]

    chunks = []
    file=[]
    page=[]
    tokens_so_far = 0
    chunk = []

    # Loop through the sentences and tokens joined together in a tuple
    for paragraph, token in zip(paragraphs, n_tokens):

        # If the number of tokens so far plus the number of tokens in the current sentence is greater
        # than the max number of tokens, then add the chunk to the list of chunks and reset
        # the chunk and tokens so far
        if tokens_so_far + token > max_tokens:
            chunks.append(". ".join(chunk) + " ")
            file.append(file_name)
            page.append(pages)
            chunk = []
            tokens_so_far = 0

        # If the number of tokens in the current sentence is greater than the max number of
        # tokens, go to the next sentence
        if token > max_tokens:
            continue

        # Otherwise, add the paragraph to the chunk and add the number of tokens to the total
        chunk.append(paragraph + " ")
        tokens_so_far += token + 1

    return chunks, file, page

def emb_with_delay(text):
    time.sleep(2)
    return openai.Embedding.create(input=text, engine='text-embedding-ada-002-rfmanrique')['data'][0]['embedding']

def process_to_embeddings():
    shortened = []
    files=[]
    pages=[]
    # Loop through the dataframe
    df = process_to_txt()

    for row in df.iterrows():
        
        # If the text is None, go to the next row
        if len(row[1]['raw_text'])==1:
            continue

        # If the number of tokens is greater than the max number of tokens, split the text into chunks
        if row[1]['n_tokens'] > max_tokens:
            short,file,page = split_into_many(row[1]['raw_text'], row[1]['name'], row[1]['page'])
            shortened+=short
            files+=file
            pages+=page

        # Otherwise, add the text to the list of shortened texts
        else:
            shortened.append( row[1]['raw_text'] )
            files.append(row[1]['name'])
            pages.append(row[1]['page'])
            
    data = {'title': files, 'page_number': pages, 'text': shortened}      
    df =  pd.DataFrame(data)
    df['n_tokens'] = df.text.apply(lambda x: len(tokenizer.encode(x)))
    df = df[df['n_tokens']>=10]
    df['embeddings'] = df.text.apply(emb_with_delay)
        
    #df['embeddings'] = df.text.apply(lambda x: openai.Embedding.create(input=x, engine='text-embedding-ada-002-rfmanrique')['data'][0]['embedding'])

    df.to_csv('embeddings_OCR.csv')
    print(df.head())
    return df

if __name__ == '__main__':
    df = process_to_embeddings()

29
                                               title  page_number  \
0  Bauer et al., 2021. Parental Attitudes toward ...            0   
1  Bauer et al., 2021. Parental Attitudes toward ...            1   
2  Bauer et al., 2021. Parental Attitudes toward ...            2   
3  Bauer et al., 2021. Parental Attitudes toward ...            3   
4  Bauer et al., 2021. Parental Attitudes toward ...            4   

                                                text  n_tokens  \
0  International  Journal  of \nEnvironmental Res...      1229   
1  Int. J. Environ. Res. Public Health 2021 ,18, ...       855   
2  Int. J. Environ. Res. Public Health 2021 ,18, ...       818   
3  Int. J. Environ. Res. Public Health 2021 ,18, ...      1145   
4  Int. J. Environ. Res. Public Health 2021 ,18, ...       834   

                                          embeddings  
0  [0.011431924067437649, 0.012565886601805687, 0...  
1  [-0.004350075963884592, 0.0003293604822829366,...  
2  [-0.0036017198581

In [21]:
df.to_csv('embeddings.csv', escapechar='\\')

In [1]:
import pandas as pd
df70 = pd.read_csv("embeddings_70.csv")
df80 = pd.read_csv("embeddings_80.csv")
df90 = pd.read_csv("embeddings_90.csv")
df00 = pd.read_csv("embeddings_00.csv")
df10 = pd.read_csv("embeddings_10.csv")
df20 = pd.read_csv("embeddings_20.csv")
df_complete = pd.concat([df70,df80,df90,df00,df10,df20])
df_complete = df_complete.drop(['Unnamed: 0'], axis=1)
df_complete.to_csv('embeddings_complete.csv', escapechar='\\')

In [2]:
import pandas as pd
df_complete = pd.read_csv("embeddings_complete.csv")
df_complete = df_complete.drop(['Unnamed: 0'], axis=1)

# Embeddings with highlighting position

In [1]:
import os
import requests
import json
import openai
import re
import PyPDF2
import pandas as pd
import tiktoken
import time
import chromadb
import numpy as np
import fitz
import re

path_dataset = '../documents'

openai.api_key = os.getenv("OpenAIKey4")
openai.api_base = "https://invuniandesai.openai.azure.com/"
openai.api_type = 'azure'
openai.api_version = '2023-05-15'

for filename in os.listdir(path_dataset):
    print(filename)

Ahmadi Vastani et al., 2016. The Effect of Experience on Recognition of Mother’s Voice in Preterm Infants.pdf
Alemdar, 2018. Effect of recorded maternal voice, breast milk odor, and incubator cover on pain and comfort during peripheral cannulation in preterm infants.pdf
Alipour, Z. et al. 2013. Effects of music on physiological and behavioral responses of premature infants- A randomized controlled trial .pdf
Amini et al., 2013. Effect of lullaby and classical music on physiologic stability of hospitalized preterm infants - A randomized trial.pdf
Arnon et al. 2014. Mohter´s singing during cangaroo care reduces axienty.pdf
Arnon, S. et al. 2006. Live Music Is Beneficial to Preterm Infants in the Neonatal Intensive Care Unit Environment.pdf
Aydin _ Yildiz, 2012. Effect of classical music on stress among preterm infants in a neonatal intensive care unit.pdf
Badr et al., 2017. Preterm infants exhibited less pain during a heel stick when they were played the same music their mothers listened

# Chroma DB

In [1]:
import chromadb

path = "C:/Users/Marine/Documents/CO_uniandes/0_Tesis/my-music-gpt/src"
chroma_client = chromadb.PersistentClient(path)
collection = chroma_client.create_collection(name="test_persist", metadata={"hnsw:space": "cosine"})


c:\Users\Marine\AppData\Local\Programs\Python\Python312\Lib\site-packages\onnxruntime\capi\onnxruntime_validation.py:26: UserWarning: Unsupported Windows version (11). ONNX Runtime supports Windows 10 and above, only.
  warnings.warn(


In [8]:
import chromadb
import numpy as np

path = "C:/Users/Marine/Documents/CO_uniandes/0_Tesis/my-music-gpt/src"
chroma_client = chromadb.PersistentClient(path)
print(chroma_client.list_collections())
collection = chroma_client.get_collection("test_persist")

df_complete['embeddings'] = df_complete['embeddings'].apply(eval).apply(np.array)

collection.add(
            embeddings=[arr.tolist() for arr in df_complete['embeddings'].to_list()],
            documents= df_complete['text'].to_list(),
            metadatas = df_complete.apply(lambda row: {"title": row['title'], "page": str(row['page_number']), "tokens": str(row['n_tokens'])}, axis=1).tolist(),
            ids=[str(i) for i in range(len(df_complete))]
        )

[Collection(name=test_persist)]
